In [346]:
import pandas as pd
import os
import datetime
import holidays
import csv
import io
from dataclasses import dataclass, asdict

In [934]:
from dataclasses import dataclass

@dataclass
class HDPowerData:
    power_plant: str = ""
    generation: float = 0
    residual_power: int = 0
    time_slot: str = 0

@dataclass
class HDClientData:
    client_id: str
    total_utility_power: float = 0  # 總市電
    total_quota: float = 0  # 總綠電轉供 (stage1 + stage2)
    time_slot: str = 0



In [1128]:
def __load_power_plant_dicts(filepath):
    power_plant_dicts = {}
    with open(filepath + os.path.sep + 'plant_information.csv', 'r', encoding="utf-8-sig") as f:
        rows = csv.reader(f, delimiter=',')
        _headers = next(rows)
        for row in rows:
            power_plant_dicts[row[0]] = {}
            # HDRE asks to round FIT to 4 digits
            #power_plant_dicts[row[0]]["surplus_price"] = float(row[1])
            power_plant_dicts[row[0]]["surplus_price"] = round(float(row[1]), 4)

            # HDRE asks to round capacity to 2 digits
            #power_plant_dicts[row[0]]["capacity"] = float(row[2])
            power_plant_dicts[row[0]]["capacity"] = round(float(row[2]), 2)
            power_plant_dicts[row[0]]["power_plant_id"] = row[3]
            power_plant_dicts[row[0]]["surplus_power"] = 0
    return power_plant_dicts

def __load_power_client_dicts(filepath):
    power_plant_dicts = {}
    with open(filepath + os.path.sep + 'client_information.csv', 'r', encoding="utf-8-sig") as f:
        rows = csv.reader(f, delimiter=',')
        _headers = next(rows)
        print(_headers)
        for row in rows:
            power_plant_dicts[row[1]] = {}
            # HDRE asks to round FIT to 4 digits
            #power_plant_dicts[row[0]]["surplus_price"] = float(row[1])
            # power_plant_dicts[row[0]]["surplus_price"] = round(float(row[1]), 4)
            #
            # # HDRE asks to round capacity to 2 digits
            # #power_plant_dicts[row[0]]["capacity"] = float(row[2])
            # power_plant_dicts[row[0]]["capacity"] = round(float(row[2]), 2)
            # power_plant_dicts[row[0]]["power_plant_id"] = row[3]
            # power_plant_dicts[row[0]]["surplus_power"] = 0
    return power_plant_dicts

@dataclass
class HDPlant:
    plant_name: str
    total_transfered: float = 0  # 總轉供
    time_slot: str = 0
    residual_power: int = 0


class HDPlantRecorder:
    def __init__(self, plant_names):
        self.hd_plant_dict = {name: HDPlant(name) for name in plant_names}

    def add_power_transfered(self, plant_name, power_transfered, time_slot, residual_power):
        obj = self.hd_plant_dict[plant_name]
        obj.total_transfered += float(power_transfered)
        obj.time_slot = time_slot
        obj.residual_power = residual_power

    def summarize(self):
        for name, obj in self.hd_plant_dict.items():
            print(asdict(obj))

@dataclass
class HDClient:
    client_id: str
    total_utility_power: float = 0  # 總市電
    total_quota: float = 0  # 總綠電轉供 (stage1 + stage2)
    time_slot: str = 0


class HDClientRecorder:
    def __init__(self, client_ids):
        self.hd_client_dict = {cid: HDClient(cid) for cid in client_ids}

    def add_quota(self, client_id, quota):
        obj = self.hd_client_dict[client_id]
        obj.total_quota += quota

    def get_total_consumption(self, client_id):
        if client_id not in self.hd_client_dict:
            return 0
        return self.hd_client_dict[client_id].total_power_consumption

    def get_total_quota(self, client_id):
        if client_id not in self.hd_client_dict:
            return 0
        return self.hd_client_dict[client_id].total_quota

    def get_hd_client(self, client_id):
        return self.hd_client_dict[client_id]

    def add_power_consumption(self, client_id, power_consumption):
        obj = self.hd_client_dict[client_id]
        obj.total_power_consumption += power_consumption

    def add_utility_power(self, client_id, utility_power):
        obj = self.hd_client_dict[client_id]
        obj.total_utility_power += utility_power

    def add_time_slot(self, client_id, time_slot):
        obj = self.hd_client_dict[client_id]
        obj.time_slot = time_slot

    def summarize(self):
        for _name, obj in self.hd_client_dict.items():
            print(asdict(obj))

def read_predefined_contracts(csv_fn):
    assert csv_fn.endswith(".csv"), f"Wrong csv_fn: {csv_fn}"
    power_plant_dict = {}
    with io.open(csv_fn, encoding="utf-8-sig") as _fp:
        reader = csv.reader(_fp, delimiter=",")
        rows = [row for row in reader]
    plant_name = rows[0][1]
    plant_generation = rows[0][2]
    docs = []
    for row in rows[1:]:
        doc = {
            'type': row[0],
            plant_name: row[1],
            plant_generation: row[2]
        }
        docs.append(doc)

    return docs

def __update_plant_recorder(plant_recorder, obj):
    for i in obj:
        plant_name = i.power_plant
        quota_sum = i.generation
        time_slot = i.time_slot
        residual_power = i.residual_power
        plant_recorder.add_power_transfered(plant_name, quota_sum, time_slot, residual_power)

def __update_client_recorder(client_record, obj):
    for i in obj:
        client_id = i.client_id
        total_utility_power = i.total_utility_power
        time_slot = i.time_slot
        total_quota = i.total_quota
        client_record.add_utility_power(client_id, total_utility_power)
        client_record.add_time_slot(client_id, time_slot)
        client_record.add_quota(client_id, total_quota)

In [1129]:
plant_ret = read_predefined_contracts('power_generation.csv')
power_plant_dict = __load_power_plant_dicts('./contract_3_5_month_minutes/')
print(power_plant_dict)
HDPlantRecorder(power_plant_dict.keys()).summarize()
power_client_dict = __load_power_client_dicts('./contract_3_5_month_minutes/')
power_client_dict
client_record = HDClientRecorder(power_client_dict.keys())
client_record.summarize()

{'電廠001': {'surplus_price': 5.0004, 'capacity': 9000.0, 'power_plant_id': '20655320001', 'surplus_power': 0}, '電廠002': {'surplus_price': 4.9552, 'capacity': 12000.0, 'power_plant_id': '20655320002', 'surplus_power': 0}}
{'plant_name': '電廠001', 'total_transfered': 0, 'time_slot': 0, 'residual_power': 0}
{'plant_name': '電廠002', 'total_transfered': 0, 'time_slot': 0, 'residual_power': 0}
['用電端', '用電端電號', '月上限', '年上限', 'RE', '最低年轉供', '綠電售價']
{'client_id': '64-67-0001-13-1', 'total_utility_power': 0, 'total_quota': 0, 'time_slot': 0}
{'client_id': '64-67-0002-13-1', 'total_utility_power': 0, 'total_quota': 0, 'time_slot': 0}
{'client_id': '64-67-0003-13-1', 'total_utility_power': 0, 'total_quota': 0, 'time_slot': 0}
{'client_id': '64-67-0004-13-1', 'total_utility_power': 0, 'total_quota': 0, 'time_slot': 0}
{'client_id': '64-67-0005-13-1', 'total_utility_power': 0, 'total_quota': 0, 'time_slot': 0}


In [1130]:
df = pd.read_csv('./contract_3_5_month_minutes/' + 'client_information.csv')
df = df.sort_values(by='綠電售價', ascending=False)
print(df['用電端電號'])
df_t = pd.read_csv('power_consumption_test.csv')
df_t = df_t.to_dict('index')
df_t

1    64-67-0002-13-1
3    64-67-0004-13-1
2    64-67-0003-13-1
4    64-67-0005-13-1
0    64-67-0001-13-1
Name: 用電端電號, dtype: object


{0: {'type': 0,
  '64-67-0001-13-1': 379638,
  '64-67-0002-13-1': 455565.6,
  '64-67-0003-13-1': 303710.4,
  '64-67-0004-13-1': 560350,
  '64-67-0005-13-1': 2186722.5},
 1: {'type': 2,
  '64-67-0001-13-1': 180670,
  '64-67-0002-13-1': 16804.0,
  '64-67-0003-13-1': 144536.0,
  '64-67-0004-13-1': 481250,
  '64-67-0005-13-1': 1040683.5}}

In [1131]:
power_plant_dict = __load_power_plant_dicts('./')
power_plant_dict
plant_record = HDPlantRecorder(power_plant_dict.keys())
plant_record.summarize()

{'plant_name': '電廠001', 'total_transfered': 0, 'time_slot': 0, 'residual_power': 0}
{'plant_name': '電廠002', 'total_transfered': 0, 'time_slot': 0, 'residual_power': 0}


In [1132]:
generation_list = []
for doc in plant_ret:
    if doc['type'] == '0':
        ret = {
            "power_plant": doc["power_plant"],
            "generation": doc['power_generation']
        }
        generation_list.append(ret)
        # print(power_plant, generation)
    elif doc['type'] == '2':
        pass


def cal_generation_0(power_list, con_list, c_list):
    rm_list = []
    for i in power_list:
        if i['power_plant'] == "電廠002":
            res = float(i.get("generation"))
            for j in c_list:
                print(j)
                res = res - float(con_list.get(0).get(j))
                docs = [
                    HDClientData(
                        client_id= j,
                        total_quota= float(con_list.get(0).get(j)),
                        total_utility_power= 0,
                        time_slot= '0'
                    )
                ]
                __update_client_recorder(client_record, docs)
                rm_list.append(j)
                if res < 0:
                    doc = [
                        HDPowerData(
                            power_plant=i['power_plant'],
                            generation=i['generation'],
                            time_slot= '0',
                            residual_power = 0
                        )
                    ]
                    __update_plant_recorder(plant_record, doc)

                    docs = [
                        HDClientData(
                            client_id= j,
                            total_quota=  res - float(con_list.get(0).get(j)),
                            total_utility_power= 0,
                            time_slot= '0'
                        )
                    ]
                    __update_client_recorder(client_record, docs)
                    print("no power")
                    break
            for r in rm_list:
                con_list.get(0).pop(r)
    return res, con_list

def cal_stage_2(power_list, power, c_list):
    record = []
    for i in power_list:
        if i['power_plant'] == "電廠001":
            res = float(i.get("generation"))
            res = res + float(power)
            doc = [
                HDPowerData(
                    power_plant=i['power_plant'],
                    generation=i['generation'],
                    time_slot= '0',
                    residual_power = 0
                )
            ]
            __update_plant_recorder(plant_record, doc)

            # docs = [
            #         HDClientData(
            #             client_id= j,
            #             total_quota=  res,
            #             total_utility_power= 0,
            #             time_slot= '0'
            #         )
            #     ]
            # __update_client_recorder(client_record, docs)
            if res < 0:
                record.append(f"{i['power_plant']}: 總轉供: {i['generation']} 總餘電: 0 時段: 0")
                break
0


0

In [1133]:
res, c_list = cal_generation_0(generation_list, df_t, df['用電端電號'])
print(c_list)
cal_stage_2(generation_list, res, c_list)

64-67-0002-13-1
64-67-0004-13-1
no power
{0: {'type': 0, '64-67-0001-13-1': 379638, '64-67-0003-13-1': 303710.4, '64-67-0005-13-1': 2186722.5}, 1: {'type': 2, '64-67-0001-13-1': 180670, '64-67-0002-13-1': 16804.0, '64-67-0003-13-1': 144536.0, '64-67-0004-13-1': 481250, '64-67-0005-13-1': 1040683.5}}


In [1025]:
plant_record.summarize()

{'plant_name': '電廠001', 'total_transfered': 447200.0, 'time_slot': '0', 'residual_power': 0}
{'plant_name': '電廠002', 'total_transfered': 559000.0, 'time_slot': '0', 'residual_power': 0}


In [1026]:
client_record.summarize()

{'client_id': '64-67-0001-13-1', 'total_utility_power': 0, 'total_quota': 0, 'time_slot': 0}
{'client_id': '64-67-0002-13-1', 'total_utility_power': 0, 'total_quota': 455565.6, 'time_slot': '0'}
{'client_id': '64-67-0003-13-1', 'total_utility_power': 0, 'total_quota': 0, 'time_slot': 0}
{'client_id': '64-67-0004-13-1', 'total_utility_power': 0, 'total_quota': -456915.6, 'time_slot': '0'}
{'client_id': '64-67-0005-13-1', 'total_utility_power': 0, 'total_quota': 0, 'time_slot': 0}
